In [1]:
print("start")
import pandas as pd
import numpy as np
from datetime import datetime
import dateutil
from sklearn import svm
from sklearn import preprocessing
import timeit

start


In [21]:
print("start")
# get data in basic table form
df = pd.read_csv("./train_downsampled.csv")
df = df.sample(frac=1)
df = df.head(100000)
df

start


/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,ip,app,device,os,channel,click_time,attributed_time,is_attributed
676259,87000651,194538,19,210,24,213,2017-11-08 05:21:50,2017-11-08 05:52:33,1
32073,46196278,181531,1,1,9,153,2017-11-07 11:35:36,NaN,0
807266,142633098,354505,10,1,7,113,2017-11-09 03:24:01,2017-11-09 03:24:35,1
398022,71035457,2600,3,1,17,424,2017-11-08 00:35:44,NaN,0
451993,82599027,3994,3,1,19,280,2017-11-08 04:06:00,NaN,0
276074,50371074,2513,12,1,23,328,2017-11-07 12:58:28,NaN,0
319083,141944517,112265,3,1,13,280,2017-11-09 03:11:03,NaN,0
833279,153176484,301018,19,16,0,213,2017-11-09 06:12:41,2017-11-09 06:13:24,1
531447,29958278,188046,45,1,37,419,2017-11-07 06:11:03,2017-11-07 06:11:14,1
72506,12286134,161021,23,1,13,153,2017-11-07 00:49:01,NaN,0


In [22]:
df['time_to_attribution'] = df.apply(lambda row: datetime.strptime(row['attributed_time'], '%Y-%m-%d %H:%M:%S') - datetime.strptime(row['click_time'], '%Y-%m-%d %H:%M:%S') if isinstance(row['attributed_time'], str) else '', axis=1)

df['hour_of_day'] = df.apply(lambda row: (datetime.strptime(row['click_time'], '%Y-%m-%d %H:%M:%S').hour + 8) % 24, axis=1)
df['day_of_week'] = df.apply(lambda row: datetime.strptime(row['click_time'], '%Y-%m-%d %H:%M:%S').weekday(), axis=1)
df['minute_of_hour'] = df.apply(lambda row: datetime.strptime(row['click_time'], '%Y-%m-%d %H:%M:%S').minute, axis=1)

print(df['day_of_week'].max())
print(df['day_of_week'].min())
print(df['hour_of_day'].max())
print(df['hour_of_day'].min())
print(df['minute_of_hour'].max())
print(df['minute_of_hour'].min())




3
0
23
0
59
0


In [23]:
df['click_time_dt'] = df.apply(lambda row: (dateutil.parser.parse(row['click_time'], dayfirst=True) - datetime.utcfromtimestamp(0)).total_seconds() , axis=1)

#clik frequency
#print(type(df['click_time']))
#ls = [df['click_time_dt'][0], df['click_time_dt'][1]]
#print(min(ls))
#print(df.isnull().sum())
"""
print(df['ip'].nunique())
ip = df.groupby('ip')['ip']
min_time = df.groupby('ip')["click_time_dt"].min()
max_time = df.groupby('ip')["click_time_dt"].max()
num_clicks = df.groupby('ip')["click_time_dt"].count()

freq = num_clicks / (max_time - min_time)
import math
f_list = []
for f in freq:
    if not math.isinf(f):
        f_list.append(f)
freq2 = sum(f_list)/len(f_list)
period = 1/freq2
arr = np.array([f_list])
stdev = np.std(arr)
outliers = arr[(arr - np.mean(arr)) > 2 * np.std(arr)]
print(outliers)
np.max(arr)
"""
#gfreq = freq.apply(math.isinf)
#print(gfreq)
#gfreq.mean()
#print(df.groupby('ip')) 
#df
#df['first_click_time'] = df.groupby('ip').min()['click_time']
#df['last_click_time'] = df.groupby('ip').click_time.max()
#df['first_last_click'] = pd.to_datetime(df['last_click_time']) - pd.to_datetime(df['first_click_time'])
#df['first_last_click_seconds'] = df['first_last_click'].dt.total_seconds()

#df['num_clicks'] = df.groupby('ip').count()
#df['click_freq'] = df['num_clicks'] / df['first_last_click_seconds']

#df


# create dataframe for is_attributed == 1
#df_ia1 = df.loc[df['time_to_attribution'] != '']

'\nprint(df[\'ip\'].nunique())\nip = df.groupby(\'ip\')[\'ip\']\nmin_time = df.groupby(\'ip\')["click_time_dt"].min()\nmax_time = df.groupby(\'ip\')["click_time_dt"].max()\nnum_clicks = df.groupby(\'ip\')["click_time_dt"].count()\n\nfreq = num_clicks / (max_time - min_time)\nimport math\nf_list = []\nfor f in freq:\n    if not math.isinf(f):\n        f_list.append(f)\nfreq2 = sum(f_list)/len(f_list)\nperiod = 1/freq2\narr = np.array([f_list])\nstdev = np.std(arr)\noutliers = arr[(arr - np.mean(arr)) > 2 * np.std(arr)]\nprint(outliers)\nnp.max(arr)\n'

In [24]:
#label is is_attributed
#fv's are ip, app, device, os, channel, click_time_dt (as a long), hour of day, and day of week
def get_fv_labels(df):
    labels = df['is_attributed']
    fv = df.drop(['Unnamed: 0','click_time', 'attributed_time', 'time_to_attribution','is_attributed'], axis=1)
    return fv, labels

# scales everything from 0 to 1
def scale_fv(mat):
    min_max_scaler = preprocessing.MinMaxScaler()
    return np.transpose(min_max_scaler.fit_transform(np.transpose(mat)))

def get_data(df, k):
    df = df.sort_values(by='click_time_dt')
    df_train = df[df['day_of_week'] != k]
    df_test = df[df['day_of_week'] == k]

    train_fv, train_labels = get_fv_labels(df_train)
    test_fv, test_labels = get_fv_labels(df_test)

    train_fv_np = train_fv.as_matrix()
    test_fv_np = test_fv.as_matrix()

    train_labels_np = train_labels.as_matrix()
    test_labels_np = test_labels.as_matrix()

    train_fv_np_sc = scale_fv(train_fv_np)
    test_fv_np_sc = scale_fv(test_fv_np)
    
    return train_fv_np_sc, train_labels_np, test_fv_np_sc, test_labels_np

In [25]:
#test_df = df[df["day_of_week"] == 3]
#df = df[df["day_of_week"] != 3]

In [26]:
from sklearn.metrics import precision_recall_fscore_support

def create_svm_model(fv, labels):
    clf = svm.SVC(C=0.1, kernel='rbf')
    # no class weight needed cuz data is downsampled to have even number of samples for each label already
    #clf = svm.SVC(C=1.0, class_weight={1:1000})
    #clf = svm.SVC(C=1.0, kernel='linear')
    
    clf.fit(fv, labels)
    return clf

import time

print("Start")
    #for k in [1, 2, 3]:
    #    print("k value: " + str(k))
train_x, train_y, test_x, test_y = get_data(df, 3)
print("training test ratio: " + str(len(train_x)/len(df)))
start = time.time()
clf = create_svm_model(train_x,train_y) 
end = time.time()
print('time',end - start)

#print(timeit.timeit(svm(test_mat_scaled)))
predict = clf.predict(test_x)

print('score',clf.score(test_x,test_y))

print('predict',predict)
print('labels',test_y)

prfs_result = precision_recall_fscore_support(test_y, predict, average='binary')
print(prfs_result)




Start
training test ratio: 0.70993
time 159.36831998825073
score 0.4952942393215431
predict [0 0 0 ... 0 0 0]
labels [1 0 1 ... 0 0 0]
(0.0, 0.0, 0.0, None)


/usr/local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [28]:
from matplotlib import pyplot as plt
from sklearn import svm

def f_importances(coef, names):
    imp = coef
    imp,names = zip(*sorted(zip(imp,names)))
    plt.barh(range(len(names)), imp, align='center')
    plt.yticks(range(len(names)), names)
    plt.show()

features_names = ['input1', 'input2']
f_importances(clf.coef_, df.columns.aslist())

AttributeError: coef_ is only available when using a linear kernel

## SVM with more features

In [25]:
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler  
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
from sklearn.cross_validation import train_test_split
import pandas as pd
from sklearn import svm
import time

In [26]:
# get data in basic table form
df = pd.read_csv("./feature_engineered_data_1.csv", delimiter='\t')
df = df.head(100000)

In [27]:
scaler = StandardScaler()  

#label is is_attributed
#fv's are ip, app, device, os, channel, click_time_dt (as a long), hour of day, and day of week
def get_fv_labels(df):
    labels = df['is_attributed']
    fv = df.drop(['Unnamed: 0', 'Unnamed: 0.1', 'attributed_time', 'time_to_attribution','is_attributed', 'click_time'], axis=1)
    return fv, labels

def get_data(df, k):
    # shuffle the data
    df = df.sample(frac=1)

    # split by day of week
    train = df[df["day_of_week"] != k]
    test = df[df["day_of_week"] == k]
    
    print("training/test split:" + str(len(train)*1.0/len(df)))
    
    train_fv, train_labels = get_fv_labels(train)
    test_fv, test_labels = get_fv_labels(test)

    print(train_fv)
    # scale feature vectors: fit only on training data
    scaler.fit(train_fv)
    train_fv_sc = scaler.transform(train_fv)
    test_fv_sc = scaler.transform(test_fv)

    return train_fv_sc, train_labels, test_fv_sc, test_labels

In [30]:
def create_svm_model(fv, labels):
    clf = svm.SVC(C=1.0, kernel='linear')
    clf.fit(fv, labels)
    return clf

train_x, train_y, test_x, test_y = get_data(df, 3)
start = time.time()
clf = create_svm_model(train_x,train_y) 
end = time.time()
print('time',end - start)

#print(timeit.timeit(svm(test_mat_scaled)))
predict = clf.predict(test_x)

print('score',clf.score(test_x,test_y))

prfs_result = precision_recall_fscore_support(test_y, predict, average='binary')
print(prfs_result)

training/test split:0.70926
           ip  app  device   os  channel  hour_of_day  day_of_week  \
72225   59404   19       0    0      343           11            1   
1735   105475   18       1   19      107           16            2   
87259  184849   35       1   19       21            9            1   
32017    5348   13       1   18      477           23            2   
31156   37145   35       1   22       21           10            1   
73724   35680   13       1   17      400           13            2   
37548  104954   28       1   13      135            2            1   
23089  105475    8       1   19      145            7            2   
38962   94146   12       1   19      178           18            2   
2003   183213   64       1    3      459           11            1   
44936  361545   29       1   17      101            0            2   
80628  116001    3       1   22      280           12            1   
89768  206979   39       1   19      101            8         

ValueError: Unsupported set of arguments: The combination of penalty='l1' and loss='squared_hinge' are not supported when dual=True, Parameters: penalty='l1', loss='squared_hinge', dual=True

In [ ]:
clf

In [ ]:
predict


In [ ]:
test_y.values



In [ ]:
print('score',clf.score(test_x,test_y))

prfs_result = precision_recall_fscore_support(test_y.values, predict, average='binary')
print(prfs_result)
print("training acuracy")
print(clf.score(train_x, train_y))

In [ ]:
list(df.columns)
test_x

In [ ]:
from matplotlib import pyplot as plt
from sklearn import svm

def f_importances(coef, names):
    imp = coef[0]
    imp,names = zip(*sorted(zip(imp,names)))
    plt.barh(range(len(names)), imp, align='center')
    plt.yticks(range(len(names)), names)
    plt.show()

fv = list(df.drop(['Unnamed: 0', 'Unnamed: 0.1', 'attributed_time', 'time_to_attribution','is_attributed', 'click_time'], axis=1).columns)
print(fv)
print(clf.coef_)
f_importances(clf.coef_, fv)



### Plot correlations and distributions for initial data set

In [ ]:
import matplotlib.pyplot as plt

#x = 'day_of_week'
data_points = ('ip', 'app', 'device', 'os', 'channel', 'hour_of_day', 'day_of_week', 'minute_of_hour')
class_data = 'is_attributed'

def print_distribution(x,y):
    print(x)
    print(df.mask(df[class_data] == 0)[x].value_counts())
    print(df.mask(df[class_data] == 1)[x].value_counts())

    print(y)
    print(df.mask(df[class_data] == 0)[y].value_counts())
    print(df.mask(df[class_data] == 1)[y].value_counts())

#print(data)
def plot(data, colors, groups, name):
    # Create plot
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)
    for data, color, group in zip(data, colors, groups):
        x, y = data
        ax.scatter(x, y, alpha=0.8, c=color, edgecolors='none', s=30, label=group)

    plt.title(name)
    #plt.legend(loc=2)
    plt.show()

def run_comparison(x, y):
    #print_distribution(x,y)
    data = ((df.mask(df[class_data] == 1)[x].values, df.mask(df[class_data] == 1)[y].values), 
           (df.mask(df[class_data] == 0)[x].values, df.mask(df[class_data] == 0)[y].values))
    colors = ("green", "blue")
    groups = ("not attributed", "attributed") 



    title = x + " vs. " + y
    plot((data[0],), (colors[0],), (groups[0],), title + ", not attributed")
    plot((data[1],), (colors[1],), (groups[1],), title + ", attributed")
    plot(data, colors, groups, title + ", attributed and not attributed")

for i in range(len(data_points)):
    for j in range(i + 1, len(data_points)):
        run_comparison(data_points[i], data_points[j])

